# Device Communication

## Overview
This document covers the basics of device communication and configuration. OpenSync devices employ USB on-the-go (OTG), so no drivers are necessary to communicate with an OpenSync device. This is beneficial as device discovery revolves arounds a computer's ability to detect a USB device that is actively connected to the computer.

## Purpose
This example demonstrates how to search, connect, and "talk" to an OpenSync device. OpenSync relies on a stable USB connection, so make sure one is present for this demonstration.

## Imports
To begin, import opensync and some of its modules.

In [1]:
from opensync import communication as comm, system

## Device Connection
We need to search for any OpenSync devices to connect to. This is performed by the function `device_comm_search`.

In [2]:
avail_ports = comm.device_comm_search()
avail_ports

['COM6']

There are two ways we can open a connection once we found the correct port to connect to. We can manually manage the resources with `device_comm_open` and `device_comm_close` or use context managers to automatically manage resources with `device_comm_managed`. Both methods will be shown in this document. 

### Manually Open Device Connection


In [3]:
# Open USB port
sync_device = comm.device_comm_open(avail_ports[0])
sync_device

Serial<id=0x1f8d772f190, open=True>(port='COM6', baudrate=19200, bytesize=8, parity='N', stopbits=1, timeout=0.25, xonxoff=False, rtscts=False, dsrdtr=False)

In [4]:
# Get device state and version
print(system.device_get_version(sync_device))
print(system.device_get_status(sync_device))

['Version: 0.0.1a']
['Run status: IDLE']


In [5]:
# Close USB port
comm.device_comm_close(sync_device)

### Open with Context Manager

In [6]:
with comm.device_comm_managed(avail_ports[0]) as sync_device:
    # Get device state and version
    print(system.device_get_version(sync_device))
    print(system.device_get_status(sync_device))

['Version: 0.0.1a']
['Run status: IDLE']


## Status Information
OpenSync allows one to poll system status information even when running a pulse sequence. Below are the most commonly used requests.

In [7]:
with comm.device_comm_managed(avail_ports[0]) as sync_device:
    print(system.device_get_version(sync_device))
    print(system.device_get_status(sync_device))
    print(system.device_get_freq(sync_device))

['Version: 0.0.1a']
['Run status: IDLE']
['pll_sys = 250000kHz', 'pll_usb = 48000kHz', 'rosc = 4970kHz', 'clk_sys = 250000kHz', 'clk_peri = 48000kHz', 'clk_usb = 48000kHz', 'clk_adc = 48000kHz', 'clk_rtc = 47kHz']


## Executing Device Parameters
Two of the more important functions for OpenSync are `device_system_fire` and `device_system_stop`. `device_system_fire` executes a pulse sequence with the current pulse and clock configrations. To prematurely stop a pulse sequence iteration, one can use `device_system_stop` and an abort signal will be sent to the OpenSync device. OpenSync allows for one to repeat a pulse sequence up to 500,000 times allowing for up to 500,000 image pairs to be collected during an experiment.

In [9]:
with comm.device_comm_managed(avail_ports[0]) as sync_device:
    # Set debug status to 1 (print internal logs then abort sequence)
    comm.device_comm_write(
        sync_device,
        'dbug 1'
    )

    # Execute with current pulse and clock configs
    print(system.device_system_fire(sync_device))
    
    # Emits error because debug level 1 aborts the sequence
    print(system.device_system_stop(sync_device))

['ok', 'Internal Message: Sequencer clock type (0=freerun; 1=triggered) = 0', 'Clock config ID: 0', 'Clock config sm: 0', 'Clock config DMA channel: 0', 'Pulse config clock divider: 1', 'Clock config clock pin: 16', 'Clock config trigger pin: 13', 'Clock instruction 0: reps=0, delay=0', 'Clock instruction 1: reps=0, delay=0', 'Clock instruction 2: reps=0, delay=0', 'Clock instruction 3: reps=0, delay=0', 'Clock instruction 4: reps=0, delay=0', 'Clock instruction 5: reps=0, delay=0', 'Clock instruction 6: reps=0, delay=0', 'Clock instruction 7: reps=0, delay=0', 'Clock instruction 8: reps=0, delay=0', 'Clock instruction 9: reps=0, delay=0', 'Clock instruction 10: reps=0, delay=0', 'Clock instruction 11: reps=0, delay=0', 'Clock instruction 12: reps=0, delay=0', 'Clock instruction 13: reps=0, delay=0', 'Clock instruction 14: reps=0, delay=0', 'Clock instruction 15: reps=0, delay=0', 'Clock instruction 16: reps=0, delay=0', 'Clock instruction 17: reps=0, delay=0', 'Clock instruction 18: r